In [1]:
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn import metrics
from contextlib import redirect_stdout
from itertools import compress
import scipy.sparse as sparse
import lightgbm as lgb
import pandas as pd
import numpy as np
import tqdm
import os
import gc
import sys
sys.path.append('../code/pipeline/')
sys.path.append('../code/utils/')
sys.path.append('../code/')
import data_pipeline as dp
import data_utils as du
import perf_utils as pu
import eval_utils as eu
import io_utils as iu
import config

In [2]:
user_one_feat_names = config.USER_SINGLE_FEAT_NAMES
user_multi_feat_names = config.USER_MULTI_FEAT_NAMES
ad_feat_names = config.AD_FEAT_NAMES

In [3]:
# ================
# Data Preparation
# ================
# instantiate data loader
emb_loaders = (dp.EmbeddingDataManager.build_data(feat_name, "min", 1) 
               for feat_name in user_multi_feat_names)
union_loader = dp.DataUnion(*emb_loaders)

with pu.profiler("loading training data"):
    cols_emb_train, X_emb_tv = union_loader.load("train")
    # X_emb_tv = sparse.csr_matrix(X_emb_tv)
    gc.collect()

print("Train Data Shape (Embedding): {}".format(X_emb_tv.shape))
print("Train Column Numbers (Embedding): {}".format(len(cols_emb_train)))

[13:04:20] Finish loading training data. △M: +15.21GB. △T: 2.6 minutes.
Train Data Shape (Embedding): (8798814, 464)
Train Column Numbers (Embedding): 464


In [4]:
df_train = du.load_raw_data("train")
y = df_train['label'].values.copy()
y = (y + 1) / 2  # -1, 1 -> 0, 1

In [5]:
# n_splits = 3
# sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=1 / 3, random_state=20180505)  # for reproducibility
# split_indices = [(train_index, valid_index) for train_index, valid_index in sss.split(df_train, y)]

n_splits = 5  # use 3 instead of 5 to save time
skf = StratifiedKFold(n_splits=n_splits)
split_indices = [(train_index, valid_index) for train_index, valid_index in skf.split(df_train, y)]

In [6]:
aids = df_train['aid'].values
with pu.profiler("splitting train/valid set"):
    train_index, valid_index = split_indices[0]
    # X_train, X_valid = X_tv[train_index, :], X_tv[valid_index, :]
    X_train, X_valid = X_emb_tv[train_index, :], X_emb_tv[valid_index, :]
    y_train, y_valid = y[train_index], y[valid_index]
    aids_train, aids_valid = aids[train_index], aids[valid_index]
    # assert X_train.shape[0] + X_valid.shape[0] == X_tv.shape[0]
    assert X_train.shape[0] + X_valid.shape[0] == X_emb_tv.shape[0]
    
    # del X_tv
    del X_emb_tv
    gc.collect()

print("Training Set Size: {}".format(X_train.shape))
print("Validation Set Size: {}".format(X_valid.shape))

[13:04:41] Finish splitting train/valid set. △M: +120.85MB. △T: 15.8 seconds.
Training Set Size: (7039050, 464)
Validation Set Size: (1759764, 464)


In [7]:
with pu.profiler("preparing LightGBM data"):
    # lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train, feature_name=cols_train)  # cause bugs I don't know how to fix
    # lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid, feature_name=cols_train)
    lgb_train = lgb.Dataset(X_train.astype(np.float32), y_train)
    lgb_valid = lgb.Dataset(X_valid.astype(np.float32), y_valid)
    gc.collect()

[13:04:53] Finish preparing LightGBM data. △M: +15.21GB. △T: 11.7 seconds.


In [8]:
with pu.profiler("releasing memory"):
    del X_train
    # del X_valid
    gc.collect()

[13:04:54] Finish releasing memory. △M: -12.17GB. △T: 1.0 seconds.


In [9]:
log_folder = os.path.join(config.LOG_DIR, 'lgbm/pipeline/0518/')
log_file = 'v4.log'
log_path = os.path.join(log_folder, log_file)
os.makedirs(log_folder, exist_ok=True)

In [10]:
# v2 parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'max_depth': 15,
    'num_leaves': 120,
    'learning_rate': 0.15,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'verbose': 0
}
num_rounds = 1000
# # v3 parameters
# params = {
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'auc',
#     'max_depth': 6,
#     'num_leaves': 64,
#     'learning_rate': 0.1,
#     'feature_fraction': 0.5,
#     'bagging_fraction': 0.5,
#     'verbose': 0
# }
# num_rounds = 5000
eval_auc = eu.build_lightgbm_online_auc_eval(aids_train, aids_valid)

with iu.DuplicatedLogger(log_path):
    lgbm = lgb.train(params,
                     lgb_train,
                     num_boost_round=num_rounds,
                     valid_sets=[lgb_train, lgb_valid], 
                     valid_names=['train', 'valid1'],
                     feval = eval_auc,
                     early_stopping_rounds=50)

[1]	train's auc: 0.626908	train's online_auc: 0.651256	valid1's auc: 0.624899	valid1's online_auc: 0.648927
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.63282	train's online_auc: 0.657958	valid1's auc: 0.630825	valid1's online_auc: 0.655437
[3]	train's auc: 0.6369	train's online_auc: 0.662645	valid1's auc: 0.634538	valid1's online_auc: 0.659702
[4]	train's auc: 0.640138	train's online_auc: 0.666116	valid1's auc: 0.637999	valid1's online_auc: 0.663337
[5]	train's auc: 0.640662	train's online_auc: 0.666575	valid1's auc: 0.638605	valid1's online_auc: 0.663906
[6]	train's auc: 0.641367	train's online_auc: 0.667285	valid1's auc: 0.639125	valid1's online_auc: 0.664211
[7]	train's auc: 0.641728	train's online_auc: 0.667583	valid1's auc: 0.639431	valid1's online_auc: 0.664625
[8]	train's auc: 0.64357	train's online_auc: 0.669447	valid1's auc: 0.641245	valid1's online_auc: 0.666447
[9]	train's auc: 0.643848	train's online_auc: 0.670025	valid1's auc: 0.641508	

[76]	train's auc: 0.687796	train's online_auc: 0.707451	valid1's auc: 0.666356	valid1's online_auc: 0.686979
[77]	train's auc: 0.688171	train's online_auc: 0.70781	valid1's auc: 0.666399	valid1's online_auc: 0.686985
[78]	train's auc: 0.688535	train's online_auc: 0.708176	valid1's auc: 0.666442	valid1's online_auc: 0.687007
[79]	train's auc: 0.688933	train's online_auc: 0.70853	valid1's auc: 0.666502	valid1's online_auc: 0.686985
[80]	train's auc: 0.689378	train's online_auc: 0.708998	valid1's auc: 0.666547	valid1's online_auc: 0.687015
[81]	train's auc: 0.689799	train's online_auc: 0.709438	valid1's auc: 0.666514	valid1's online_auc: 0.686983
[82]	train's auc: 0.690206	train's online_auc: 0.709831	valid1's auc: 0.66668	valid1's online_auc: 0.687085
[83]	train's auc: 0.690513	train's online_auc: 0.710133	valid1's auc: 0.666723	valid1's online_auc: 0.687198
[84]	train's auc: 0.691038	train's online_auc: 0.710571	valid1's auc: 0.666828	valid1's online_auc: 0.687163
[85]	train's auc: 0.69

[151]	train's auc: 0.71299	train's online_auc: 0.732186	valid1's auc: 0.667179	valid1's online_auc: 0.68708
[152]	train's auc: 0.713207	train's online_auc: 0.73235	valid1's auc: 0.667168	valid1's online_auc: 0.687139
[153]	train's auc: 0.713449	train's online_auc: 0.732568	valid1's auc: 0.667169	valid1's online_auc: 0.687164
[154]	train's auc: 0.713768	train's online_auc: 0.732904	valid1's auc: 0.667172	valid1's online_auc: 0.687119
[155]	train's auc: 0.714029	train's online_auc: 0.733119	valid1's auc: 0.667184	valid1's online_auc: 0.687094
[156]	train's auc: 0.714307	train's online_auc: 0.733373	valid1's auc: 0.667183	valid1's online_auc: 0.687115
[157]	train's auc: 0.714532	train's online_auc: 0.733631	valid1's auc: 0.667191	valid1's online_auc: 0.687139
[158]	train's auc: 0.714893	train's online_auc: 0.734051	valid1's auc: 0.667197	valid1's online_auc: 0.687113
[159]	train's auc: 0.715283	train's online_auc: 0.734469	valid1's auc: 0.667224	valid1's online_auc: 0.687138
[160]	train's

In [11]:
# these will cause LightGBM bug I don't know how to fix. Just skip it
log_file = 'v4.feature_importance.csv'
log_path = os.path.join(log_folder, log_file)

# df_feature_importance = pd.DataFrame({"feature": cols_train, "importance": lgbm.feature_importance()})
df_feature_importance = pd.DataFrame({"feature": cols_emb_train, "importance": lgbm.feature_importance()})
df_feature_importance = df_feature_importance.sort_values("importance", ascending=False)
df_feature_importance.to_csv(log_path, index=False)
df_feature_importance.head(30)

,feature,importance
147,kw2_embedding_20,200
151,kw2_embedding_24,161
165,kw2_embedding_38,155
167,kw2_embedding_40,148
169,kw2_embedding_42,148
145,kw2_embedding_18,146
139,kw2_embedding_12,141
143,kw2_embedding_16,141
315,topic2_embedding_38,132
44,interest2_embedding_11,132


In [12]:
with pu.profiler("preparing valid set"):
    cols_emb_train, X_emb_tv = union_loader.load("train")
    X_valid = X_emb_tv[valid_index, :]
    del X_emb_tv
    gc.collect()

[13:40:31] Finish preparing valid set. △M: -39.09MB. △T: 2.2 minutes.


In [13]:
with pu.profiler("making prediction on validation set"):
    df_valid = df_train.iloc[valid_index]
    proba_valid = lgbm.predict(X_valid.astype(np.float32))

log_file = 'v4.online_auc.csv'
log_path = os.path.join(log_folder, log_file)
df_score = eu.online_auc(df_valid['aid'], y_valid, proba_valid, ret_verbose=True)
df_score.rename(columns={'selector': 'aid'}, inplace=True)
df_score = df_score[['aid', 'auc']]  # sort columns
df_score = df_score.sort_values("auc", ascending=False)
df_score.to_csv(log_path, index=False)

[13:40:37] Finish making prediction on validation set. △M: +80.0KB. △T: 6.7 seconds.


In [14]:
online_auc = df_score['auc'].mean()
simple_auc = metrics.roc_auc_score(y_valid, proba_valid)
print("Online AUC: {:.6f}".format(online_auc))
print("Simple AUC: {:.6f}".format(simple_auc))

Online AUC: 0.687605
Simple AUC: 0.667572


In [15]:
with pu.profiler("cleaning memory"):
    del lgb_train
    del lgb_valid
    # del X_train
    del X_valid
    gc.collect()

[13:40:40] Finish cleaning memory. △M: -7.21GB. △T: 0.8 seconds.


In [16]:
with pu.profiler("loading testing data"):
    cols_test, X_test = union_loader.load("test1")
    X_test = sparse.csr_matrix(X_test)
    gc.collect()

print("Test Data Shape: {}".format(X_test.shape))
# print("Test Column Numbers: {}".format(len(cols_test)))

[13:42:05] Finish loading testing data. △M: +7.84GB. △T: 1.4 minutes.
Test Data Shape: (2265989, 464)


In [17]:
df_test = du.load_raw_data("test")
X_test = X_test.astype(np.float32)

with pu.profiler("making prediction on testing set"):
    proba_test = lgbm.predict(X_test)
    assert len(proba_test.shape) == 1
    assert proba_test.shape[0] == df_test.shape[0]

[13:42:18] Finish making prediction on testing set. △M: +0B. △T: 5.1 seconds.


In [18]:
subm_folder = '../subm/lgbm/0518_v4'
subm_file = 'submission.csv'
subm_path = os.path.join(subm_folder, subm_file)
os.makedirs(subm_folder, exist_ok=True)

subm = df_test.copy()
subm["score"] = proba_test
subm.to_csv(subm_path, index=False)